In [2]:
import yaml
import os

dualview_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
device= "cuda" if os.environ.get("CUDA_VISIBLE_DEVICES") else "cpu"
data_root="<data_root>"

In [3]:
base_config={
    "validation_size": 2000,
}

training_dict_MNIST ={
    "epochs": 200,
    "loss": "cross_entropy",
    "lr": 5e-3,
    "momentum": 0.9,
    "optimizer": "sgd",
    "scheduler": "constant",
    "weight_decay": 0,
    "augmentation": "crop"
}

training_dict_CIFAR ={
    "epochs": 190,
    "loss": "cross_entropy",
    "lr": 1e-3,
    "momentum": 0.9,
    "optimizer": "sgd",
    "scheduler": "constant",
    "weight_decay": 0.01,
    "augmentation": "crop_flip"
}

training_dict_AWA ={
    "epochs": 50,
    "loss": "cross_entropy",
    "lr": 0.01,
    "momentum": 0.9,
    "optimizer": "sgd",
    "scheduler": "annealing",
    "weight_decay": 0,
    "augmentation": "flip"
}

training_dict_empty ={
    "epochs": None,
    "loss": None,
    "lr": None,
    "momentum": None,
    "optimizer": None,
    "scheduler": None,
    "weight_decay": None,
    "augmentation": None
}

In [4]:
def save_config(config, config_name):
    config = {k: v for k, v in config.items() if v is not None}
    path = f"lds/{config['dataset_name']}"
    os.makedirs(path, exist_ok=True)
    with open(f"{path}/{config_name}.yaml", "w") as outfile:
        yaml.dump(config, outfile, default_flow_style=False)

def create_config_local(config, config_name):
    config['device'] = device
    config['data_root'] = data_root
    config['save_dir'] = f"{dualview_path}/cache/{config['dataset_name']}/lds"
    config['cache_dir'] = f"{dualview_path}/cache/{config['dataset_name']}/lds"
    if config['metric'] == 'mark':
        config['model_path']=f"{dualview_path}/checkpoints/{config['dataset_name']}/{config['dataset_type']}/{config['model_name']}_{config['dataset_type']}/{config['dataset_name']}_{config['model_name']}_best"
    else:
        config['model_path'] = None

    save_config(config, f"{int(config['sample_nr']):02d}_{config_name}")

In [5]:
dsname_list = ["MNIST", "CIFAR", "AWA"]
metric_list = ["lds_cache"]
sample_nr_list = list(range(100))

model_dict = {"MNIST": "basic_conv", "CIFAR": "resnet18", "AWA": "resnet50"}
num_classes_dict = {"MNIST": 10, "CIFAR": 10, "AWA": 50}


In [6]:
from copy import deepcopy

for dsname in dsname_list:
    print(f"Creating config files for dataset {dsname}...")
    for metric in metric_list:
        for sample_nr in sample_nr_list:
            config = deepcopy(base_config)
            config['dataset_type']='std'
            config['dataset_name'] = dsname
            config['model_name'] = model_dict[dsname]
            config['num_classes'] = num_classes_dict[dsname]
            config['metric'] = metric
            config['class_groups'] = None
            config['sample_nr'] = sample_nr
            config['batch_size'] = 16
            config.update(globals()[f'training_dict_{dsname}'])
            config_filename = f"lds_cache_{dsname}"
            create_config_local(config, config_filename)

Creating config files for dataset MNIST...
Creating config files for dataset CIFAR...
Creating config files for dataset AWA...
